# 1. About the notebooks

The notebooks located in the folder: **misc/notebooks/** can be used as tutorials for understanding the working and the capabilities of the **QRBD** software package.<

# 2. About the myQLM and Qaptiva Access environment

The ecosystem of quantum simulation tools of **Qaptiva's Eviden** can be split into 2 different well-differentiated (and interconnected) tools:

* **myQLM** (https://myqlm.github.io/index.html#): is the quantum software stack developed by Eviden, for writing, simulating, optimizing and executing quantum programs. This software is a freeware Python package which comes with interoperability connectors. With this tool only two basic simulators are available to the user: **CLinAlg** and **PyLinAlg**.

* **Qaptiva 800 series**: is the Quantum Appliance Toolset developed by Eviden. This software product extends the capabilities of myQLM, adding more functionalities, more emulators and the ability to execute on real quantum hardware. In the case of the emulators, it expands the maximum number of simulated qubits beyond 25. This *Toolset* was designed to be executed in the **EVIDEN Quantum Learning Machine** (https://atos.net/en/solutions/quantum-learning-machine). In CESGA facilities, for example, the **QLM30** is available. This machine guarantees the possibility of simulating circuits up to 30 qubits, but in some cases, it can simulate larger cases. Also, it includes other advanced simulators a linear-algebra-based quantum emulator **LinAlg** or a Matrix Product State **MPS** one. Additionally, this software comes with many extra functionalities like circuit rewriting and optimization tools and the possibility for configuring and simulating noisy Quantum Processing Units using the **Noisy Quantum Emulator** (**NoisyQProc** object).

These tools have the same semantics to create quantum circuits, so the user can write a quantum program using **myQLM** and optimize, rewrite and execute the circuit using the **Qaptiva 800 series**. Additionally, a third tool called **Qaptiva Access** allows the user to submit **myQLM** heavy computation to a remote **Qaptiva** machine (like the one installed in CESGA facilities)


## 3. The qpu package

The **qpu** package was developed to handle all the functionalities of these tools. This package allows easy configuration of the different functionalities available for the **myQLM** and for the **Qaptiva 800 series**.

The main module of the **qpu/** package is the **qpu/select_qpu** one. This module implements a wrapper function called *select_qpu*. The input is a Python dictionary, whose scheme is shown here:

    qpu_cfg = {
        "qpu_type": str,
        "qpu_name": str,
        "kak_compiler": str,
        "sim_method": {
            "sim_method": str,
            "bond_dimension": int,
            "n_samples": int
        }, 
        "t_gate_1qb": int,
        "t_gate_2qbs": int,
        "t_readout": int,
        "depol_channel" : {
            "active": bool,
            "error_gate_1qb": float,
            "error_gate_2qbs": float
        },
        "idle": {
            "amplitude_damping": bool,
            "dephasing_channel": bool,
            "t1": int,
            "t2": int
        },
        "meas": {
            "active": bool,
            "readout_error": float
        }
    }
    
    
The Python dictionary allows the user to configure 3 different types of QPUs:

* Ideal noiseless **QPU**: In this case, the following strings can be provided to the *qpu_type* key of the Python dictionary:
    * *python*: for using **PyLinalg** lineal algebra simulator based on Python
    * *c*: for using **CLinalg** lineal algebra simulator based on C
    * *linalg*:  for using the **LinAlg**, a lineal algebra simulator only available in **EVIDEN QLM hardware**. To use it, the user must be locally connected to a **EVIDEN QLM hardware**.
    * *mps*: for using the *Matrix Product State* simulator only available in **EVIDEN QLM hardware**. To use it, the user must be locally connected to a **EVIDEN QLM hardware**.
    * *qlmass_linalg*: for submitting the circuit(s) **LinAlg** using the **Qaptiva Access** tool (in this case the user needs access to a remote **EVIDEN QLM hardware**)
    * *qlmass_mps*: for submitting the circuit(s) to a *Matrix Product State* simulator using the **Qaptiva Access** tool  (in this case the user needs access to a remote **EVIDEN QLM hardware**)
* Ideal noiseless **QPU** with circuit rewriting. In this case, some **EVIDEN** tools are used for building a ideal **QPU** that rewrittes the circuit of the input job (see section 3.1 Circuit Rewriting). The simulation will be ideal and the **LinAlg** will be used. To use it, the user must be locally connected to a **EVIDEN QLM hardware**. For using it the *ideal* string should be provided to the *qpu_type* key of the Python dictionary.
* Noisy **QPU**. In this case,  the user can configure a noisy **QPU** (see 3.2 Nosiy Model and ). For using it the *noisy* string should be provided to the *qpu_type* key of the Python dictionary. 



### 3.1 Circuit Rewriting

**BE AWARE!! To use this type of QPU the user should be connected directly to a Qactiva hardware**

The **qpu** package allows the user to build a **QPU** object with rewrites the circuit of the jobs before sending it to the **QPU**.  Two rewriting **Qaptiva 800** functionalities are enabled in this case:

* **KAK compression** plugin: merge consecutive one qubit gates into a temporary unitary matrix, and decompose this temporary matrix using a selected pattern. The pattern can be selected by providing the following strings to the *kak_compiler* key of the Python dictionary
    * ZXZ: the unitary matrix is decomposed using the following pattern: $R_z-R_x-R_z$.
    * XZX: the unitary matrix is decomposed using the pattern: $R_x-R_z-R_x$.    
    * ZYZ: the unitary matrix is decomposed using the pattern: $R_z-R_y-R_z$.        
    * ions: the unitary matrix is decomposed using the pattern: $R_z-R_x\left(\frac{\pi}{2}\right)-R_z-R_x\left(\frac{\pi}{2}\right)-R_z$.
    
* **Toffoli Rewriter**: using the **Pattern Manager** plugin from QLM, all the *Toffoli* gates of the input quantum circuit will be decomposed in 1 and 2-qubit gates.Ç

### 3.2 Noisy Model

**BE AWARE!! To use this type of QPU the user should be connected directly to a Qactiva hardware**

The **select_qpu** function from **qpu/select_qpu** module allows the user to configure noisy simulation, using different **Qaptiva 800 series** functionalities and plugins. To activate these capabilities, the string *noisy* should be provided to the *qpu_type* key of the Python dictionary. To execute noisy simulations, two main ingredients are mandatory: a noisy model and the simulation method. Here the configuration of the noisy model is provided and section 3.3 provides the configuration for the simulation method is provided.

The noisy simulation model is composed of three different, configurable, parts:

* Depolarization channel: in this noisy model any unitary gate can have a probability of not being applied correctly. To enable it, the **depol_chanel** sub dictionary should be modified:
    * The *active* key should be set to True.
    * The *error_gate_1qb* should be set to the desired failure probability of one qubit.
    * The *error_gate_2qbs* should be set to the desired failure probability of two-qubit gates.
*  Idle noise: this part models the behaviour of the qubits during the time that no gate is being applied to them (idle time). The subdictionary key **idle** configures the two different types of idle noises that can be applied:
    * Amplitude Damping channel: this is related to the decoherence time $T_1$ of the qubits. To enable it:
        * The *amplitude_damping* key should be set to True. 
        * The $T_1$ time (nanoseconds) should be provided to the **t1** key.
    * Phase Damping channel: this is related to the decoherence time $T_2$ of the qubits. To enable it:
        * The **dephasing_channel** key should be set to True.
        * The $T_2$ time (nanoseconds) should be provided to the **t2** key. This case needs the **amplitude_damping** enabled and configurated.
* Readout error: models the error in the measurement. To configure it, the sub-dictionary **meas** should be used:
    * To enable it, the corresponding **active** key should be set to True. 
    * The specific measurement error value should provided to the **readout_error** key.

### 3. Simulation method

Once the noise model is configured the simulation method should be set. This is done in the sub-dictionary *sim_method*. For selecting the simulation method the **sim_method** key from the *sim_method* sub-dictionary is used. The following methods can be used:

* deterministic simulation: in this case, the complete density matrix evolution for the quantum circuit is simulated. This simulation has a high storage cost but there is no statistical error. To use it the following strings can be provided to the *sim_method* key: *deterministic* or *deterministic-vectorized*. This type of simulation can be used when the number of qubits of the quantum circuits is not too large (for the **QLM 30** in CESGA facilities the thumb rule is no more than 15 qubits).
* stochastic simulation: this kind of simulation performs a stochastic sampling over all possible trajectories. To use it the string **stochastic** should be provided to the **sim_method** key. The storage cost is lower than in the deterministic but it has some statistical error given by the number of trajectories. The number of trajectories should be provided to the sub-dictionary using the **n_samples** key of the *sim_method* sub-dictionary.
* Matrix Product Operator (*MPO*) simulation: in this case, the simulation uses tensor networks and matrix product state techniques for approximating the final result. To use it the string **mpo** should be provided to the **sim_method** key. An additional parameter is the bond dimension of the *MPO* that should be provided (as an integer) to the key **bond_dimension** of the *sim_method* sub-dictionary.

## 4. JSON configuration files

Several **JSON** files with different **QPU**s configuration are located inside the **qpu/** folder:

* qpu_ideal.json : **JSON** for configuring ideal **QPU**s.
* qpu_noisy_deterministic.json : **JSON** for configuring noisy **QPU**s using the *deterministic* simulation method.
* qpu_noisy_stochastic.json : **JSON** for configuring noisy **QPU**s using the *stochastic* simulation method.
* qpu_noisy_mpo.json : **JSON** for configuring noisy **QPU**s using the *MPO* simulation method.